In [55]:
#!pip install bs4 requests

import requests
import bs4
import json
import re
import time
import random

# original idea: https://gist.github.com/CharlieTLe/9272de175edb85b07e332c2108288451
# see also: https://github.com/GodLesZ/1001tracklists-scraper/blob/master/lib/scraper.js
#
# to launch: https://mybinder.org/v2/gh/pestrela/music_scripts/master


def list_get_random_element(l):
    return random.choice(l)

def delay_get_random(base=60):
    l = []
    for i in range(1,11,3):
        l.append(base+i)
        
    print(l)    
    return list_get_random_element(l)



In [56]:
  
def html_get_header():

    requests_hdr_list = [ 
     {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) ' 
       'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.37'
       },
     {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.12'
           '(KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
           'Accept-Language': 'en-US,en;q=0.8'
           },
           
    {
       'User-Agent': 'Mozilla/5.0 (Macintosh; Intel'
       'Mac OS X 10_12_1) AppleWebKit/537.37 (KHTML, '
       'like Gecko) Chrome/55.0.2883.95 Safari/537.36'
       },
       
    {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.12 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection' : 'keep-alive'},
       
       
   { 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6)'
    ' AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.144 Safari/537.36'
   },
    
    {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
        'referrer': 'https://google.com',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Pragma': 'no-cache',
    },
    
     
       
    ]
    
    return list_get_random_element(requests_hdr_list)



def get_recursive_sets(main_url, limit=5, start=None, grep=None):
    urls = get_latest_urls(main_url=main_url, limit=limit, start=start, grep=grep)
    
    # default delay for limited queries
    if limit:
        delay = 3
    
    for url in urls:
        if limit is None:
            delay = delay_get_random(30)
        
        get_one_set(url)
        time.sleep(delay)
        
        
        
def get_html(url, verbose=True):
    headers =  html_get_header()
    r = requests.get(url, headers=headers)
    if r.status_code != 200:
        raise Exception()
        
    soup = bs4.BeautifulSoup(r.text, 'html.parser')

    if verbose:
        print("\n\n%s\n\n\n" % (url))
    return soup        
        
    
def get_latest_urls(main_url, limit=5, start=None, grep=None, verbose=True):
    soup = get_html(main_url)

    urls = []
    
    i = 1
    for setLink in soup.find_all('div', class_="tlLink"):
        link = 'https://www.1001tracklists.com' + setLink.find('a').get('href')
        print("%s\n\n" % (link))
        
        if grep and (grep not in link):
            if verbose:
                print("ignoring: %s", (link))
            continue
            
        #print(link)
        if start and (i<start):
            continue
        
        urls.append(link)
        i = i + 1
        if limit and (i > limit):
            break
    
    print("\n\n")
    
    #print(urls)
    
    print('\n\n')
    return urls

def dump_list(l):
    print(*["%s\n" % (i) for i in l ])
        
        
def get_one_set(url, debug=False, detect_problem=True):

    soup = get_html(url)
    
    i = 1
    for a in soup.find_all('tr'): #, id="tlp_3943296"):
        names=[]
        cues=[]
        #print(a)

        for name in a.find_all('div', {"class": "tlToogleData"}):
            name = name.text.strip()
            #print(name)
            if "\n" in name:
                name = name.splitlines()[0]

            if "correct label is" in name:
                continue

            names.append(name)

        for cue in a.find_all('div', {"class": "cueValueField"}):
            cue = cue.text.strip()
            cues.append(cue)

        if names == []:
            continue

        i = i + 1
        #print(names)
        #print(cues)

        name = " ".join(names)
        if cues == []:
            cue = ""
        else:
            cue = " ".join(cues)
            cue = "%s - " % (cue)

        line = "%s%s" % (cue, name)
        print(line)
        
    if detect_problem and i <= 4:
        raise Exception
            
            
    delay = 2
    #if limit and (limit > 20):
    #    delay = 3
        
    time.sleep(delay)

        
        
def do_work(data, limit=5, start=None, grep=None, token=None, oper=None):
    """
    what:
        index_sets:       single URL with a list -> get individual set urls
        recursive_sets:   single URL with a list -> all sets details
        isolated_sets:    input is a LIST of urls -> get set details
    
    """
    if token != 1:
        print("ERROR: Bad token")
        return

    ###
    for line in data.splitlines():
        line = line.strip()
        if not line:
            continue
            
        if oper == "index_sets":
            get_latest_urls(line, limit=limit, start=start)

        elif oper == "recursive_sets":
            get_recursive_sets(line, limit=limit, start=start, grep=grep)

        elif oper == "isolated_sets":
            get_one_set(line)

        else:
            raise ValueError("Unknown value", what)
    

In [58]:
data="""

https://www.1001tracklists.com/source/rch80m/a-state-of-trance-festival/index.html

"""

token=2

start=None
limit=None
grep=None #"2019"

#oper="index_sets"
oper="recursive_sets"
#oper="isolated_sets"

do_work(data, limit=limit, start=start, grep=grep, token=token, oper=oper)

print('\n\nALL DONE \n\n')



https://www.1001tracklists.com/source/rch80m/a-state-of-trance-festival/index.html



https://www.1001tracklists.com/tracklist/jzdql5t/elevven-progressive-stage-a-state-of-trance-festival-jaarbeurs-utrecht-netherlands-2020-02-15.html


https://www.1001tracklists.com/tracklist/v76w1r9/ritmo-psy-stage-a-state-of-trance-festival-950-jaarbeurs-utrecht-netherlands-2020-02-15.html


https://www.1001tracklists.com/tracklist/1jnp9kd9/ahmed-helmy-road-to-1000-stage-a-state-of-trance-festival-950-jaarbeurs-utrecht-netherlands-2020-02-15.html


https://www.1001tracklists.com/tracklist/1zkj2mjt/beatsole-road-to-1000-stage-a-state-of-trance-festival-950-jaarbeurs-utrecht-netherlands-2020-02-15.html


https://www.1001tracklists.com/tracklist/2br45hl9/drym-road-to-1000-stage-a-state-of-trance-festival-950-jaarbeurs-utrecht-netherlands-2020-02-15.html


https://www.1001tracklists.com/tracklist/1gt1sdz9/tempo-giusto-road-to-1000-stage-a-state-of-trance-festival-950-jaarbeurs-utrecht-netherlands-2020-

In [33]:
html_get_header()

{'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.12(KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
 'Accept-Language': 'en-US,en;q=0.8'}